In [1]:
import json
import shutil

import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import tiledb
import pdal
from pybabylonjs import Show as show

In [37]:
#!wget -nc "https://github.com/PDAL/data/blob/master/workshop/autzen.laz?raw=true" -O "autzen.laz"

In [4]:
#try:
#    shutil.rmtree("autzen")
#    pass
#except:
#    pass

In [36]:
pipeline1 = (
  pdal.Reader("autzen.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen1",chunk_size=10000000)
)

count = pipeline1.execute()  

In [37]:
pipeline2 = (
  pdal.Reader("autzen.laz") | 
  pdal.Writer.tiledb(array_name="autzen2",x_tile_size=10, y_tile_size=10, z_tile_size=1,chunk_size=10000000)
)

count = pipeline2.execute() 

In [11]:
pipeline3 = (
  pdal.Reader("autzen.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen3",chunk_size=10000000,data_tile_capacity=100000)
)

count = pipeline3.execute()  

In [9]:
pipeline4 = (
  pdal.Reader("autzen.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen4",chunk_size=100000,data_tile_capacity=100000)
)

count = pipeline4.execute()  

In [11]:
pipeline5 = (
  pdal.Reader("autzen.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen5",chunk_size=100000)
)

count = pipeline5.execute()  

## Point cloud

In [2]:
%%time
with tiledb.open("autzen1") as arr:
    df = pd.DataFrame(arr[636800:637800, 851000:853000, 406.14:615.26])

df.head()

CPU times: user 4.94 s, sys: 913 ms, total: 5.85 s
Wall time: 1.31 s


,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,UserData,PointSourceId,GpsTime,Red,Green,Blue,X,Y,Z
0,1,1,1,1,0,2,-1.0,122,7331,248289.538944,111,112,98,637475.45,851804.30,415.49
1,1,1,1,1,0,2,10.0,122,7332,248680.116914,114,119,113,637174.99,852014.95,415.49
2,1,1,1,0,0,2,-12.0,124,7330,247564.723623,134,136,113,637371.69,851840.65,415.51
3,1,1,1,0,0,2,0.0,124,7331,248288.150659,138,145,124,637244.90,851873.25,415.51
4,1,1,1,1,0,2,-11.0,124,7330,247565.570015,112,114,108,637169.48,851998.29,415.51


In [3]:
data = {
    'X': df['X'],
    'Y': df['Y'],
    'Z': df['Z'],
    'Red': df['Red'] / 255.0,
    'Green': df['Green'] / 255.0,
    'Blue': df['Blue'] / 255.0
}

In [4]:
show.from_dict(data=data,
                      style = 'pointcloud',
                        width = 800,
                        height = 600,
                        z_scale = .2,
                        wheel_precision = 50)

BabylonPC(value={'style': 'pointcloud', 'width': 800, 'height': 600, 'z_scale': 0.2, 'wheel_precision': 50, 'e…

## MBRS

### Default sorting

In [5]:
show.from_array(array='autzen1',
                               style='mbrs',
                               width=800,
                               height=600,
                               z_scale = 0.5,
                               wheel_precision = 50)

BabylonMBRS(value={'style': 'mbrs', 'width': 800, 'height': 600, 'z_scale': 0.5, 'wheel_precision': 50, 'exten…

### Optimized sorting - visual tuning - 2.5 x faster loading of data from array

In [10]:
show.from_array(array='autzen5',
                               style='mbrs',
                               width=800,
                               height=600,
                               z_scale = 0.2)

BabylonMBRS(value={'style': 'mbrs', 'width': 800, 'height': 600, 'z_scale': 0.2, 'wheel_precision': 50, 'exten…

In [11]:
%%time
with tiledb.open("autzen1") as arr:
    df = pd.DataFrame(arr[636800:637800, 851000:853000, 406.14:615.26])

CPU times: user 5.61 s, sys: 1.27 s, total: 6.88 s
Wall time: 1.41 s


In [12]:
%%time
with tiledb.open("autzen5") as arr:
    df = pd.DataFrame(arr[636800:637800, 851000:853000, 406.14:615.26])

CPU times: user 1.73 s, sys: 634 ms, total: 2.36 s
Wall time: 612 ms


## 4D 

In [13]:
%%time
with tiledb.open("autzen5") as arr:
    df = pd.DataFrame(arr[636800:637800, 851000:853000, 406.14:615.26])

data = {
    'X': df['X'],
    'Y': df['Y'],
    'Z': df['Z'],
    'Red': df['Red'] / 255.0,
    'Green': df['Green'] / 255.0,
    'Blue': df['Blue'] / 255.0,
    'GpsTime': df['GpsTime']}

CPU times: user 1.72 s, sys: 657 ms, total: 2.38 s
Wall time: 576 ms


In [15]:
show.from_dict(data=data,
                      style = 'pointcloud',
                        width = 800,
                        height = 600,
                        z_scale = .2,
                        time = True)

BabylonPC(value={'style': 'pointcloud', 'width': 800, 'height': 600, 'z_scale': 0.2, 'wheel_precision': 50, 'e…